# Personal Life Automation Agent System\n## Google ADK Capstone Project\n\n**Complete Self-Contained Implementation**\n\nRun all cells sequentially to launch the Gradio interface!\n\n### Features:\n- 🍽️ Meal Planning\n- 🛒 Shopping Lists\n- ✈️ Travel Planning\n- 🤖 Multi-Agent System

In [ ]:
!pip install gradio python-dotenv -q

## Data Models

In [ ]:
"""Data models for the Personal Life Automation Agent System."""
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from datetime import date, datetime
from enum import Enum


class MealType(str, Enum):
    """Enum for meal types."""
    BREAKFAST = "breakfast"
    LUNCH = "lunch"
    DINNER = "dinner"
    SNACK = "snack"


class TaskStatus(str, Enum):
    """Enum for task status."""
    RUNNING = "running"
    PAUSED = "paused"
    COMPLETED = "completed"
    FAILED = "failed"


@dataclass
class UserProfile:
    """User profile with preferences and restrictions."""
    user_id: str
    dietary_restrictions: List[str] = field(default_factory=list)
    allergies: List[str] = field(default_factory=list)
    cuisine_preferences: List[str] = field(default_factory=list)
    pantry_items: List[str] = field(default_factory=list)
    budget_preferences: Dict = field(default_factory=dict)
    travel_interests: List[str] = field(default_factory=list)
    
    def validate(self) -> bool:
        """Validate user profile data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.user_id:
            raise ValueError("user_id is required")
        return True


@dataclass
class Ingredient:
    """Ingredient with quantity and unit."""
    name: str
    quantity: float
    unit: str
    
    def validate(self) -> bool:
        """Validate ingredient data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.name:
            raise ValueError("Ingredient name is required")
        if self.quantity <= 0:
            raise ValueError("Ingredient quantity must be positive")
        if not self.unit:
            raise ValueError("Ingredient unit is required")
        return True


@dataclass
class Meal:
    """Meal with recipe details."""
    meal_type: str  # breakfast, lunch, dinner, snack
    recipe_id: str
    recipe_name: str
    ingredients: List[Ingredient]
    instructions: str
    prep_time: int  # minutes
    cook_time: int  # minutes
    
    def validate(self) -> bool:
        """Validate meal data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if self.meal_type not in [mt.value for mt in MealType]:
            raise ValueError(f"Invalid meal_type: {self.meal_type}")
        if not self.recipe_id:
            raise ValueError("recipe_id is required")
        if not self.recipe_name:
            raise ValueError("recipe_name is required")
        if not self.ingredients:
            raise ValueError("Meal must have at least one ingredient")
        if not self.instructions:
            raise ValueError("instructions are required")
        if self.prep_time < 0:
            raise ValueError("prep_time must be non-negative")
        if self.cook_time < 0:
            raise ValueError("cook_time must be non-negative")
        
        # Validate all ingredients
        for ingredient in self.ingredients:
            ingredient.validate()
        
        return True
    
    def total_time(self) -> int:
        """Calculate total cooking time.
        
        Returns:
            Total time in minutes
        """
        return self.prep_time + self.cook_time


@dataclass
class MealPlan:
    """Meal plan for a time period."""
    plan_id: str
    user_id: str
    start_date: date
    end_date: date
    meals: List[Meal]
    total_recipes: int
    
    def validate(self) -> bool:
        """Validate meal plan data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.plan_id:
            raise ValueError("plan_id is required")
        if not self.user_id:
            raise ValueError("user_id is required")
        if self.start_date > self.end_date:
            raise ValueError("start_date must be before or equal to end_date")
        if not self.meals:
            raise ValueError("Meal plan must have at least one meal")
        if self.total_recipes != len(set(m.recipe_id for m in self.meals)):
            raise ValueError("total_recipes must match unique recipe count")
        
        # Validate all meals
        for meal in self.meals:
            meal.validate()
        
        return True
    
    def duration_days(self) -> int:
        """Calculate duration in days.
        
        Returns:
            Number of days in the meal plan
        """
        return (self.end_date - self.start_date).days + 1


@dataclass
class ShoppingItem:
    """Shopping list item."""
    name: str
    quantity: float
    unit: str
    category: str
    estimated_price: float = 0.0
    
    def validate(self) -> bool:
        """Validate shopping item data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.name:
            raise ValueError("Item name is required")
        if self.quantity <= 0:
            raise ValueError("Item quantity must be positive")
        if not self.unit:
            raise ValueError("Item unit is required")
        if not self.category:
            raise ValueError("Item category is required")
        if self.estimated_price < 0:
            raise ValueError("estimated_price must be non-negative")
        return True


@dataclass
class ShoppingList:
    """Shopping list with categorized items."""
    list_id: str
    user_id: str
    created_date: date
    items: List[ShoppingItem]
    categories: Dict[str, List[ShoppingItem]]
    estimated_total: float
    
    def validate(self) -> bool:
        """Validate shopping list data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.list_id:
            raise ValueError("list_id is required")
        if not self.user_id:
            raise ValueError("user_id is required")
        if not self.items:
            raise ValueError("Shopping list must have at least one item")
        if self.estimated_total < 0:
            raise ValueError("estimated_total must be non-negative")
        
        # Validate all items
        for item in self.items:
            item.validate()
        
        # Validate categories contain all items
        categorized_items = []
        for category_items in self.categories.values():
            categorized_items.extend(category_items)
        
        if len(categorized_items) != len(self.items):
            raise ValueError("All items must be categorized")
        
        return True
    
    def item_count(self) -> int:
        """Get total number of items.
        
        Returns:
            Number of items in the list
        """
        return len(self.items)


@dataclass
class Activity:
    """Activity for a trip itinerary."""
    name: str
    description: str
    duration: int  # minutes
    estimated_cost: float = 0.0
    location: str = ""
    
    def validate(self) -> bool:
        """Validate activity data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.name:
            raise ValueError("Activity name is required")
        if self.duration <= 0:
            raise ValueError("Activity duration must be positive")
        if self.estimated_cost < 0:
            raise ValueError("estimated_cost must be non-negative")
        return True


@dataclass
class Restaurant:
    """Restaurant recommendation."""
    name: str
    cuisine_type: str
    estimated_cost: float
    location: str = ""
    
    def validate(self) -> bool:
        """Validate restaurant data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.name:
            raise ValueError("Restaurant name is required")
        if not self.cuisine_type:
            raise ValueError("cuisine_type is required")
        if self.estimated_cost < 0:
            raise ValueError("estimated_cost must be non-negative")
        return True


@dataclass
class Accommodation:
    """Accommodation details."""
    name: str
    type: str  # hotel, airbnb, hostel, etc.
    location: str
    cost_per_night: float
    total_cost: float
    amenities: List[str] = field(default_factory=list)
    
    def validate(self) -> bool:
        """Validate accommodation data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.name:
            raise ValueError("Accommodation name is required")
        if not self.type:
            raise ValueError("Accommodation type is required")
        if not self.location:
            raise ValueError("Accommodation location is required")
        if self.cost_per_night < 0:
            raise ValueError("cost_per_night must be non-negative")
        if self.total_cost < 0:
            raise ValueError("total_cost must be non-negative")
        return True


@dataclass
class DayPlan:
    """Daily itinerary for a trip."""
    day_number: int
    date: date
    activities: List[Activity]
    meals: List[Restaurant]
    notes: str = ""
    
    def validate(self) -> bool:
        """Validate day plan data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if self.day_number <= 0:
            raise ValueError("day_number must be positive")
        
        # Validate all activities
        for activity in self.activities:
            activity.validate()
        
        # Validate all restaurants
        for restaurant in self.meals:
            restaurant.validate()
        
        return True
    
    def total_estimated_cost(self) -> float:
        """Calculate total estimated cost for the day.
        
        Returns:
            Total cost including activities and meals
        """
        activity_cost = sum(a.estimated_cost for a in self.activities)
        meal_cost = sum(m.estimated_cost for m in self.meals)
        return activity_cost + meal_cost


@dataclass
class TripPlan:
    """Complete trip plan with itinerary."""
    trip_id: str
    user_id: str
    destination: str
    start_date: date
    end_date: date
    accommodation: Accommodation
    itinerary: List[DayPlan]
    estimated_cost: float
    
    def validate(self) -> bool:
        """Validate trip plan data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.trip_id:
            raise ValueError("trip_id is required")
        if not self.user_id:
            raise ValueError("user_id is required")
        if not self.destination:
            raise ValueError("destination is required")
        if self.start_date > self.end_date:
            raise ValueError("start_date must be before or equal to end_date")
        if not self.itinerary:
            raise ValueError("Trip must have at least one day plan")
        if self.estimated_cost < 0:
            raise ValueError("estimated_cost must be non-negative")
        
        # Validate accommodation
        self.accommodation.validate()
        
        # Validate all day plans
        for day_plan in self.itinerary:
            day_plan.validate()
        
        return True
    
    def duration_days(self) -> int:
        """Calculate trip duration in days.
        
        Returns:
            Number of days in the trip
        """
        return (self.end_date - self.start_date).days + 1


@dataclass
class AgentState:
    """Agent execution state for pause/resume."""
    task_id: str
    agent_type: str
    status: str  # running, paused, completed, failed
    current_step: int
    total_steps: int
    context: Dict
    created_at: datetime
    updated_at: datetime
    
    def validate(self) -> bool:
        """Validate agent state data.
        
        Returns:
            True if valid, raises ValueError otherwise
        """
        if not self.task_id:
            raise ValueError("task_id is required")
        if not self.agent_type:
            raise ValueError("agent_type is required")
        if self.status not in [s.value for s in TaskStatus]:
            raise ValueError(f"Invalid status: {self.status}")
        if self.current_step < 0:
            raise ValueError("current_step must be non-negative")
        if self.total_steps <= 0:
            raise ValueError("total_steps must be positive")
        if self.current_step > self.total_steps:
            raise ValueError("current_step cannot exceed total_steps")
        return True
    
    def progress_percentage(self) -> float:
        """Calculate progress percentage.
        
        Returns:
            Progress as a percentage (0-100)
        """
        if self.total_steps == 0:
            return 0.0
        return (self.current_step / self.total_steps) * 100


## Memory Bank

In [ ]:
"""Memory Bank implementation for persistent user preferences and feedback."""
import json
from pathlib import Path
from typing import Any, Optional
from datetime import datetime, timezone


class MemoryBank:
    """Manages persistent storage of user preferences, feedback, and pantry inventory."""
    
    def __init__(self, storage_path: Path):
        """Initialize Memory Bank with storage path.
        
        Args:
            storage_path: Directory path for storing user data files
        """
        self.storage_path = Path(storage_path)
        self.storage_path.mkdir(parents=True, exist_ok=True)
    
    def _get_user_file(self, user_id: str) -> Path:
        """Get the file path for a user's data.
        
        Args:
            user_id: User identifier
            
        Returns:
            Path to user's JSON data file
        """
        return self.storage_path / f"{user_id}.json"
    
    def _load_user_data(self, user_id: str) -> dict:
        """Load user data from file.
        
        Args:
            user_id: User identifier
            
        Returns:
            Dictionary containing user data, or empty dict if file doesn't exist
        """
        user_file = self._get_user_file(user_id)
        if user_file.exists():
            with open(user_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {
            "preferences": {},
            "feedback": [],
            "pantry": []
        }
    
    def _save_user_data(self, user_id: str, data: dict) -> None:
        """Save user data to file.
        
        Args:
            user_id: User identifier
            data: Dictionary containing user data to save
        """
        user_file = self._get_user_file(user_id)
        with open(user_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    
    async def save_preference(self, user_id: str, key: str, value: Any) -> None:
        """Save a user preference.
        
        Args:
            user_id: User identifier
            key: Preference key (e.g., 'dietary_restrictions', 'cuisine_preferences')
            value: Preference value (can be any JSON-serializable type)
        """
        data = self._load_user_data(user_id)
        data["preferences"][key] = value
        self._save_user_data(user_id, data)
    
    async def get_preference(self, user_id: str, key: str) -> Optional[Any]:
        """Retrieve a user preference.
        
        Args:
            user_id: User identifier
            key: Preference key
            
        Returns:
            Preference value, or None if not found
        """
        data = self._load_user_data(user_id)
        return data["preferences"].get(key)
    
    async def get_all_preferences(self, user_id: str) -> dict:
        """Retrieve all preferences for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            Dictionary of all user preferences
        """
        data = self._load_user_data(user_id)
        return data["preferences"]
    
    async def update_feedback(self, user_id: str, item_id: str, rating: float) -> None:
        """Store user feedback/rating for an item.
        
        Args:
            user_id: User identifier
            item_id: Identifier for the rated item (e.g., recipe_id, trip_id)
            rating: Rating value (typically 1-5)
        """
        data = self._load_user_data(user_id)
        
        feedback_entry = {
            "item_id": item_id,
            "rating": rating,
            "timestamp": datetime.now(timezone.utc).isoformat()
        }
        
        data["feedback"].append(feedback_entry)
        self._save_user_data(user_id, data)
    
    async def get_feedback_history(self, user_id: str) -> list:
        """Retrieve all feedback history for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            List of feedback entries with item_id, rating, and timestamp
        """
        data = self._load_user_data(user_id)
        return data["feedback"]
    
    async def save_pantry_items(self, user_id: str, items: list[str]) -> None:
        """Save pantry inventory for a user.
        
        Args:
            user_id: User identifier
            items: List of pantry item names
        """
        data = self._load_user_data(user_id)
        data["pantry"] = items
        self._save_user_data(user_id, data)
    
    async def get_pantry_items(self, user_id: str) -> list[str]:
        """Retrieve pantry inventory for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            List of pantry item names
        """
        data = self._load_user_data(user_id)
        return data["pantry"]
    
    async def add_pantry_item(self, user_id: str, item: str) -> None:
        """Add a single item to user's pantry.
        
        Args:
            user_id: User identifier
            item: Pantry item name to add
        """
        data = self._load_user_data(user_id)
        if item not in data["pantry"]:
            data["pantry"].append(item)
            self._save_user_data(user_id, data)
    
    async def remove_pantry_item(self, user_id: str, item: str) -> None:
        """Remove a single item from user's pantry.
        
        Args:
            user_id: User identifier
            item: Pantry item name to remove
        """
        data = self._load_user_data(user_id)
        if item in data["pantry"]:
            data["pantry"].remove(item)
            self._save_user_data(user_id, data)


## Session Manager

In [ ]:
"""Session Manager implementation for conversation state management."""
import uuid
from typing import Optional, Dict, List
from datetime import datetime, timezone
from dataclasses import dataclass, field, asdict
import json


@dataclass
class Message:
    """Represents a single message in a conversation."""
    role: str  # 'user' or 'assistant'
    content: str
    timestamp: str = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())


@dataclass
class Session:
    """Represents a user session with conversation history."""
    session_id: str
    user_id: str
    messages: List[Message] = field(default_factory=list)
    created_at: str = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    updated_at: str = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    metadata: Dict = field(default_factory=dict)
    
    def to_dict(self) -> dict:
        """Convert session to dictionary."""
        return {
            "session_id": self.session_id,
            "user_id": self.user_id,
            "messages": [asdict(msg) for msg in self.messages],
            "created_at": self.created_at,
            "updated_at": self.updated_at,
            "metadata": self.metadata
        }
    
    @classmethod
    def from_dict(cls, data: dict) -> "Session":
        """Create session from dictionary."""
        messages = [Message(**msg) for msg in data.get("messages", [])]
        return cls(
            session_id=data["session_id"],
            user_id=data["user_id"],
            messages=messages,
            created_at=data.get("created_at", datetime.now(timezone.utc).isoformat()),
            updated_at=data.get("updated_at", datetime.now(timezone.utc).isoformat()),
            metadata=data.get("metadata", {})
        )


class InMemorySessionService:
    """In-memory storage for sessions."""
    
    def __init__(self):
        """Initialize in-memory session storage."""
        self._sessions: Dict[str, Session] = {}
    
    def save(self, session: Session) -> None:
        """Save a session to memory."""
        self._sessions[session.session_id] = session
    
    def get(self, session_id: str) -> Optional[Session]:
        """Retrieve a session from memory."""
        return self._sessions.get(session_id)
    
    def delete(self, session_id: str) -> None:
        """Delete a session from memory."""
        if session_id in self._sessions:
            del self._sessions[session_id]
    
    def list_by_user(self, user_id: str) -> List[Session]:
        """List all sessions for a user."""
        return [s for s in self._sessions.values() if s.user_id == user_id]


class SessionManager:
    """Manages user sessions and conversation history."""
    
    def __init__(self, session_service: InMemorySessionService, memory_bank=None):
        """Initialize Session Manager.
        
        Args:
            session_service: Service for storing/retrieving sessions
            memory_bank: Optional MemoryBank for loading user preferences
        """
        self.session_service = session_service
        self.memory_bank = memory_bank
    
    async def create_session(self, user_id: str) -> str:
        """Create a new session for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            Session ID
        """
        session_id = str(uuid.uuid4())
        session = Session(session_id=session_id, user_id=user_id)
        
        # Load user preferences if memory bank is available
        if self.memory_bank:
            preferences = await self.memory_bank.get_all_preferences(user_id)
            session.metadata["preferences"] = preferences
        
        self.session_service.save(session)
        return session_id
    
    async def get_session(self, session_id: str) -> Optional[Session]:
        """Retrieve a session.
        
        Args:
            session_id: Session identifier
            
        Returns:
            Session object or None if not found
        """
        return self.session_service.get(session_id)
    
    async def update_session(self, session_id: str, messages: List[Message]) -> None:
        """Update session with new messages.
        
        Args:
            session_id: Session identifier
            messages: List of messages to add to the session
        """
        session = self.session_service.get(session_id)
        if session:
            session.messages.extend(messages)
            session.updated_at = datetime.now(timezone.utc).isoformat()
            self.session_service.save(session)
    
    async def add_message(self, session_id: str, role: str, content: str) -> None:
        """Add a single message to a session.
        
        Args:
            session_id: Session identifier
            role: Message role ('user' or 'assistant')
            content: Message content
        """
        message = Message(role=role, content=content)
        await self.update_session(session_id, [message])
    
    async def close_session(self, session_id: str) -> None:
        """Close and delete a session.
        
        Args:
            session_id: Session identifier
        """
        self.session_service.delete(session_id)
    
    async def get_conversation_history(self, session_id: str) -> List[Message]:
        """Get conversation history for a session.
        
        Args:
            session_id: Session identifier
            
        Returns:
            List of messages in the session
        """
        session = self.session_service.get(session_id)
        return session.messages if session else []
    
    async def list_user_sessions(self, user_id: str) -> List[Session]:
        """List all sessions for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            List of sessions for the user
        """
        return self.session_service.list_by_user(user_id)
    
    async def update_session_metadata(self, session_id: str, key: str, value: any) -> None:
        """Update session metadata.
        
        Args:
            session_id: Session identifier
            key: Metadata key
            value: Metadata value
        """
        session = self.session_service.get(session_id)
        if session:
            session.metadata[key] = value
            session.updated_at = datetime.now(timezone.utc).isoformat()
            self.session_service.save(session)


## State Persistence

In [ ]:
"""State Persistence implementation for pause/resume functionality."""
import json
from pathlib import Path
from typing import Optional, List, Dict, Any
from datetime import datetime, timezone


class StatePersistence:
    """Manages persistent storage of agent execution state for pause/resume."""
    
    def __init__(self, storage_path: Path):
        """Initialize State Persistence with storage path.
        
        Args:
            storage_path: Directory path for storing state files
        """
        self.storage_path = Path(storage_path)
        self.storage_path.mkdir(parents=True, exist_ok=True)
    
    def _get_state_file(self, task_id: str) -> Path:
        """Get the file path for a task's state.
        
        Args:
            task_id: Task identifier
            
        Returns:
            Path to task's state JSON file
        """
        return self.storage_path / f"{task_id}.json"
    
    def _get_user_index_file(self, user_id: str) -> Path:
        """Get the index file path for a user's tasks.
        
        Args:
            user_id: User identifier
            
        Returns:
            Path to user's task index file
        """
        return self.storage_path / f"user_{user_id}_index.json"
    
    def _load_user_index(self, user_id: str) -> Dict[str, Any]:
        """Load user's task index.
        
        Args:
            user_id: User identifier
            
        Returns:
            Dictionary mapping task_id to task metadata
        """
        index_file = self._get_user_index_file(user_id)
        if index_file.exists():
            with open(index_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save_user_index(self, user_id: str, index: Dict[str, Any]) -> None:
        """Save user's task index.
        
        Args:
            user_id: User identifier
            index: Dictionary mapping task_id to task metadata
        """
        index_file = self._get_user_index_file(user_id)
        with open(index_file, 'w', encoding='utf-8') as f:
            json.dump(index, f, indent=2, ensure_ascii=False)
    
    async def save_state(self, task_id: str, state: Dict[str, Any]) -> None:
        """Save agent execution state for a task.
        
        Args:
            task_id: Task identifier
            state: Dictionary containing task state (must include 'user_id')
        """
        if 'user_id' not in state:
            raise ValueError("State must include 'user_id' field")
        
        # Add metadata
        state_with_metadata = {
            **state,
            'task_id': task_id,
            'saved_at': datetime.now(timezone.utc).isoformat()
        }
        
        # Save state file
        state_file = self._get_state_file(task_id)
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(state_with_metadata, f, indent=2, ensure_ascii=False)
        
        # Update user index
        user_id = state['user_id']
        index = self._load_user_index(user_id)
        index[task_id] = {
            'task_id': task_id,
            'agent_type': state.get('agent_type', 'unknown'),
            'status': state.get('status', 'paused'),
            'saved_at': state_with_metadata['saved_at']
        }
        self._save_user_index(user_id, index)
    
    async def load_state(self, task_id: str) -> Optional[Dict[str, Any]]:
        """Load agent execution state for a task.
        
        Args:
            task_id: Task identifier
            
        Returns:
            Dictionary containing task state, or None if not found
        """
        state_file = self._get_state_file(task_id)
        if state_file.exists():
            with open(state_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return None
    
    async def delete_state(self, task_id: str) -> None:
        """Delete agent execution state for a task.
        
        Args:
            task_id: Task identifier
        """
        # Load state to get user_id before deleting
        state = await self.load_state(task_id)
        
        # Delete state file
        state_file = self._get_state_file(task_id)
        if state_file.exists():
            state_file.unlink()
        
        # Update user index if we have user_id
        if state and 'user_id' in state:
            user_id = state['user_id']
            index = self._load_user_index(user_id)
            if task_id in index:
                del index[task_id]
                self._save_user_index(user_id, index)
    
    async def list_paused_tasks(self, user_id: str) -> List[Dict[str, Any]]:
        """List all paused tasks for a user.
        
        Args:
            user_id: User identifier
            
        Returns:
            List of task metadata dictionaries
        """
        index = self._load_user_index(user_id)
        return list(index.values())
    
    async def task_exists(self, task_id: str) -> bool:
        """Check if a task state exists.
        
        Args:
            task_id: Task identifier
            
        Returns:
            True if task state exists, False otherwise
        """
        state_file = self._get_state_file(task_id)
        return state_file.exists()
    
    async def update_task_status(self, task_id: str, status: str) -> None:
        """Update the status of a paused task.
        
        Args:
            task_id: Task identifier
            status: New status (e.g., 'paused', 'running', 'completed', 'failed')
        """
        state = await self.load_state(task_id)
        if state:
            state['status'] = status
            state['updated_at'] = datetime.now(timezone.utc).isoformat()
            await self.save_state(task_id, state)


## Recipe Tool

In [ ]:
"""Recipe Database MCP Tool for meal planning."""
import logging
import time
from typing import List, Dict, Optional
from datetime import datetime

logger = logging.getLogger(__name__)


class RecipeDatabaseTool:
    """MCP tool interface for accessing recipe databases.
    
    This is a stub implementation with mock data for development.
    In production, this would connect to a real recipe API via MCP protocol.
    """
    
    def __init__(self, api_key: Optional[str] = None, max_retries: int = 3):
        """Initialize the recipe database tool.
        
        Args:
            api_key: API key for recipe service (optional for mock)
            max_retries: Maximum number of retry attempts for failed requests
        """
        self.api_key = api_key
        self.max_retries = max_retries
        self.call_count = 0  # For testing tool invocation
        logger.info("RecipeDatabaseTool initialized")
    
    async def search_recipes(
        self,
        query: str = "",
        cuisine: Optional[str] = None,
        dietary_restrictions: Optional[List[str]] = None,
        max_results: int = 10
    ) -> List[Dict]:
        """Search for recipes matching criteria.
        
        Args:
            query: Search query string
            cuisine: Cuisine type filter (e.g., "italian", "mexican")
            dietary_restrictions: List of dietary restrictions (e.g., ["vegetarian", "gluten-free"])
            max_results: Maximum number of results to return
            
        Returns:
            List of recipe dictionaries with id, name, cuisine, ingredients, etc.
            
        Raises:
            Exception: If API call fails after retries
        """
        self.call_count += 1
        logger.info(f"Searching recipes: query='{query}', cuisine={cuisine}, restrictions={dietary_restrictions}")
        
        # Simulate API call with retry logic
        for attempt in range(self.max_retries):
            try:
                # Mock recipe data
                recipes = self._get_mock_recipes()
                
                # Filter by cuisine
                if cuisine:
                    cuisine_filtered = [r for r in recipes if r['cuisine'].lower() == cuisine.lower()]
                    if cuisine_filtered:
                        recipes = cuisine_filtered
                
                # Filter by dietary restrictions (flexible matching)
                if dietary_restrictions:
                    filtered = [r for r in recipes if self._matches_restrictions(r, dietary_restrictions)]
                    # If no exact matches, return all recipes (flexible fallback)
                    if filtered:
                        recipes = filtered
                    else:
                        logger.warning(f"No recipes match restrictions {dietary_restrictions}, returning all available")
                
                # Filter by query
                if query:
                    query_filtered = [r for r in recipes if query.lower() in r['name'].lower()]
                    if query_filtered:
                        recipes = query_filtered
                
                return recipes[:max_results]
                
            except Exception as e:
                logger.warning(f"Recipe search attempt {attempt + 1} failed: {e}")
                if attempt < self.max_retries - 1:
                    # Exponential backoff
                    wait_time = 2 ** attempt
                    logger.info(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    logger.error("Recipe search failed after all retries")
                    raise
    
    async def get_recipe_details(self, recipe_id: str) -> Dict:
        """Get detailed information for a specific recipe.
        
        Args:
            recipe_id: Unique recipe identifier
            
        Returns:
            Recipe dictionary with complete details including instructions
            
        Raises:
            ValueError: If recipe not found
            Exception: If API call fails
        """
        self.call_count += 1
        logger.info(f"Fetching recipe details for: {recipe_id}")
        
        recipes = self._get_mock_recipes()
        recipe = next((r for r in recipes if r['id'] == recipe_id), None)
        
        if not recipe:
            raise ValueError(f"Recipe not found: {recipe_id}")
        
        return recipe
    
    async def get_random_recipes(
        self,
        count: int = 5,
        cuisine: Optional[str] = None,
        dietary_restrictions: Optional[List[str]] = None
    ) -> List[Dict]:
        """Get random recipes matching criteria.
        
        Args:
            count: Number of random recipes to return
            cuisine: Optional cuisine filter
            dietary_restrictions: Optional dietary restriction filters
            
        Returns:
            List of random recipe dictionaries
        """
        self.call_count += 1
        logger.info(f"Getting {count} random recipes")
        
        import random
        recipes = self._get_mock_recipes()
        
        # Apply filters (flexible matching)
        if cuisine:
            cuisine_filtered = [r for r in recipes if r['cuisine'].lower() == cuisine.lower()]
            if cuisine_filtered:
                recipes = cuisine_filtered
        
        if dietary_restrictions:
            filtered = [r for r in recipes if self._matches_restrictions(r, dietary_restrictions)]
            # If no exact matches, use all recipes (flexible fallback)
            if filtered:
                recipes = filtered
            else:
                logger.warning(f"No recipes match restrictions {dietary_restrictions}, using all available")
        
        # Return random selection
        return random.sample(recipes, min(count, len(recipes)))
    
    def _matches_restrictions(self, recipe: Dict, restrictions: List[str]) -> bool:
        """Check if recipe matches dietary restrictions.
        
        Args:
            recipe: Recipe dictionary
            restrictions: List of dietary restrictions
            
        Returns:
            True if recipe matches all restrictions
        """
        recipe_tags = [tag.lower() for tag in recipe.get('dietary_tags', [])]
        return all(restriction.lower() in recipe_tags for restriction in restrictions)
    
    def _get_mock_recipes(self) -> List[Dict]:
        """Get mock recipe data for development.
        
        Returns:
            List of mock recipe dictionaries
        """
        return [
            {
                'id': 'recipe_001',
                'name': 'Vegetarian Pasta Primavera',
                'cuisine': 'Italian',
                'dietary_tags': ['vegetarian', 'dairy'],
                'prep_time': 15,
                'cook_time': 20,
                'servings': 4,
                'ingredients': [
                    {'name': 'pasta', 'quantity': 1, 'unit': 'lb'},
                    {'name': 'bell peppers', 'quantity': 2, 'unit': 'whole'},
                    {'name': 'zucchini', 'quantity': 1, 'unit': 'whole'},
                    {'name': 'olive oil', 'quantity': 2, 'unit': 'tbsp'},
                    {'name': 'parmesan cheese', 'quantity': 0.5, 'unit': 'cup'},
                ],
                'instructions': '1. Cook pasta according to package. 2. Sauté vegetables in olive oil. 3. Combine pasta and vegetables. 4. Top with parmesan.',
                'rating': 4.5
            },
            {
                'id': 'recipe_002',
                'name': 'Grilled Chicken Tacos',
                'cuisine': 'Mexican',
                'dietary_tags': ['gluten-free'],
                'prep_time': 20,
                'cook_time': 15,
                'servings': 4,
                'ingredients': [
                    {'name': 'chicken breast', 'quantity': 1.5, 'unit': 'lb'},
                    {'name': 'corn tortillas', 'quantity': 12, 'unit': 'whole'},
                    {'name': 'avocado', 'quantity': 2, 'unit': 'whole'},
                    {'name': 'lime', 'quantity': 2, 'unit': 'whole'},
                    {'name': 'cilantro', 'quantity': 0.5, 'unit': 'cup'},
                ],
                'instructions': '1. Season and grill chicken. 2. Warm tortillas. 3. Slice chicken and avocado. 4. Assemble tacos with toppings.',
                'rating': 4.7
            },
            {
                'id': 'recipe_003',
                'name': 'Vegan Buddha Bowl',
                'cuisine': 'Asian',
                'dietary_tags': ['vegan', 'gluten-free'],
                'prep_time': 25,
                'cook_time': 30,
                'servings': 2,
                'ingredients': [
                    {'name': 'quinoa', 'quantity': 1, 'unit': 'cup'},
                    {'name': 'chickpeas', 'quantity': 1, 'unit': 'can'},
                    {'name': 'sweet potato', 'quantity': 1, 'unit': 'whole'},
                    {'name': 'kale', 'quantity': 2, 'unit': 'cups'},
                    {'name': 'tahini', 'quantity': 0.25, 'unit': 'cup'},
                ],
                'instructions': '1. Cook quinoa. 2. Roast sweet potato and chickpeas. 3. Massage kale. 4. Assemble bowl and drizzle with tahini.',
                'rating': 4.8
            },
            {
                'id': 'recipe_004',
                'name': 'Classic Beef Burger',
                'cuisine': 'American',
                'dietary_tags': [],
                'prep_time': 10,
                'cook_time': 15,
                'servings': 4,
                'ingredients': [
                    {'name': 'ground beef', 'quantity': 1.5, 'unit': 'lb'},
                    {'name': 'burger buns', 'quantity': 4, 'unit': 'whole'},
                    {'name': 'lettuce', 'quantity': 4, 'unit': 'leaves'},
                    {'name': 'tomato', 'quantity': 1, 'unit': 'whole'},
                    {'name': 'cheddar cheese', 'quantity': 4, 'unit': 'slices'},
                ],
                'instructions': '1. Form beef into patties. 2. Grill burgers to desired doneness. 3. Toast buns. 4. Assemble burgers with toppings.',
                'rating': 4.6
            },
            {
                'id': 'recipe_005',
                'name': 'Mediterranean Salad',
                'cuisine': 'Mediterranean',
                'dietary_tags': ['vegetarian', 'gluten-free'],
                'prep_time': 15,
                'cook_time': 0,
                'servings': 4,
                'ingredients': [
                    {'name': 'mixed greens', 'quantity': 6, 'unit': 'cups'},
                    {'name': 'cucumber', 'quantity': 1, 'unit': 'whole'},
                    {'name': 'cherry tomatoes', 'quantity': 2, 'unit': 'cups'},
                    {'name': 'feta cheese', 'quantity': 0.5, 'unit': 'cup'},
                    {'name': 'olives', 'quantity': 0.5, 'unit': 'cup'},
                    {'name': 'olive oil', 'quantity': 3, 'unit': 'tbsp'},
                ],
                'instructions': '1. Chop vegetables. 2. Combine all ingredients in bowl. 3. Drizzle with olive oil and lemon juice. 4. Toss and serve.',
                'rating': 4.4
            },
            {
                'id': 'recipe_006',
                'name': 'Teriyaki Salmon',
                'cuisine': 'Japanese',
                'dietary_tags': ['gluten-free', 'pescatarian'],
                'prep_time': 10,
                'cook_time': 20,
                'servings': 2,
                'ingredients': [
                    {'name': 'salmon fillets', 'quantity': 2, 'unit': 'whole'},
                    {'name': 'teriyaki sauce', 'quantity': 0.25, 'unit': 'cup'},
                    {'name': 'rice', 'quantity': 1, 'unit': 'cup'},
                    {'name': 'broccoli', 'quantity': 2, 'unit': 'cups'},
                    {'name': 'sesame seeds', 'quantity': 1, 'unit': 'tbsp'},
                ],
                'instructions': '1. Marinate salmon in teriyaki. 2. Cook rice. 3. Bake salmon at 400°F for 15 min. 4. Steam broccoli. 5. Serve with sesame seeds.',
                'rating': 4.9
            }
        ]


## Pricing Tool

In [ ]:
"""Pricing API MCP Tool for shopping list price estimates."""
import logging
import time
from typing import List, Dict, Optional

logger = logging.getLogger(__name__)


class PricingAPITool:
    """MCP tool interface for accessing grocery pricing APIs.
    
    This is a stub implementation with mock data for development.
    In production, this would connect to a real pricing API via MCP protocol.
    """
    
    def __init__(self, api_key: Optional[str] = None, max_retries: int = 3):
        """Initialize the pricing API tool.
        
        Args:
            api_key: API key for pricing service (optional for mock)
            max_retries: Maximum number of retry attempts for failed requests
        """
        self.api_key = api_key
        self.max_retries = max_retries
        self.call_count = 0  # For testing tool invocation
        self.rate_limit_count = 0  # For testing rate limit handling
        logger.info("PricingAPITool initialized")
    
    async def get_item_price(
        self,
        item_name: str,
        quantity: float = 1.0,
        unit: str = "unit",
        store: Optional[str] = None
    ) -> Dict:
        """Get price estimate for a single item.
        
        Args:
            item_name: Name of the grocery item
            quantity: Quantity needed
            unit: Unit of measurement
            store: Optional store preference
            
        Returns:
            Dictionary with item_name, quantity, unit, price_per_unit, total_price
            
        Raises:
            Exception: If API call fails after retries
        """
        self.call_count += 1
        logger.info(f"Getting price for: {item_name} ({quantity} {unit})")
        
        # Simulate API call with retry logic
        for attempt in range(self.max_retries):
            try:
                # Simulate rate limiting occasionally
                if self.rate_limit_count > 0:
                    self.rate_limit_count -= 1
                    raise Exception("Rate limit exceeded")
                
                price_per_unit = self._get_mock_price(item_name)
                total_price = price_per_unit * quantity
                
                return {
                    'item_name': item_name,
                    'quantity': quantity,
                    'unit': unit,
                    'price_per_unit': price_per_unit,
                    'total_price': round(total_price, 2),
                    'store': store or 'Generic Store',
                    'currency': 'USD'
                }
                
            except Exception as e:
                logger.warning(f"Price lookup attempt {attempt + 1} failed: {e}")
                if attempt < self.max_retries - 1:
                    # Exponential backoff
                    wait_time = 2 ** attempt
                    logger.info(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    logger.error("Price lookup failed after all retries")
                    # Return fallback price instead of failing
                    return {
                        'item_name': item_name,
                        'quantity': quantity,
                        'unit': unit,
                        'price_per_unit': 5.0,  # Fallback price
                        'total_price': round(5.0 * quantity, 2),
                        'store': 'Estimated',
                        'currency': 'USD',
                        'note': 'Estimated price - API unavailable'
                    }
    
    async def get_bulk_prices(
        self,
        items: List[Dict],
        store: Optional[str] = None
    ) -> List[Dict]:
        """Get price estimates for multiple items.
        
        Args:
            items: List of item dictionaries with name, quantity, unit
            store: Optional store preference
            
        Returns:
            List of price dictionaries
        """
        self.call_count += 1
        logger.info(f"Getting bulk prices for {len(items)} items")
        
        results = []
        for item in items:
            price_info = await self.get_item_price(
                item_name=item['name'],
                quantity=item.get('quantity', 1.0),
                unit=item.get('unit', 'unit'),
                store=store
            )
            results.append(price_info)
        
        return results
    
    async def compare_stores(
        self,
        item_name: str,
        quantity: float = 1.0,
        unit: str = "unit"
    ) -> List[Dict]:
        """Compare prices across different stores.
        
        Args:
            item_name: Name of the grocery item
            quantity: Quantity needed
            unit: Unit of measurement
            
        Returns:
            List of price dictionaries from different stores
        """
        self.call_count += 1
        logger.info(f"Comparing prices for: {item_name}")
        
        stores = ['Walmart', 'Target', 'Whole Foods', 'Kroger']
        results = []
        
        base_price = self._get_mock_price(item_name)
        
        for store in stores:
            # Add some variation to prices
            import random
            variation = random.uniform(0.9, 1.1)
            price_per_unit = round(base_price * variation, 2)
            
            results.append({
                'item_name': item_name,
                'quantity': quantity,
                'unit': unit,
                'price_per_unit': price_per_unit,
                'total_price': round(price_per_unit * quantity, 2),
                'store': store,
                'currency': 'USD'
            })
        
        # Sort by total price
        results.sort(key=lambda x: x['total_price'])
        return results
    
    def simulate_rate_limit(self, count: int = 1):
        """Simulate rate limiting for testing.
        
        Args:
            count: Number of requests to fail with rate limit
        """
        self.rate_limit_count = count
        logger.info(f"Simulating rate limit for next {count} requests")
    
    def _get_mock_price(self, item_name: str) -> float:
        """Get mock price for an item.
        
        Args:
            item_name: Name of the item
            
        Returns:
            Mock price per unit
        """
        # Mock price database
        prices = {
            'pasta': 2.99,
            'bell peppers': 1.49,
            'zucchini': 1.99,
            'olive oil': 8.99,
            'parmesan cheese': 6.99,
            'chicken breast': 5.99,
            'corn tortillas': 3.49,
            'avocado': 1.99,
            'lime': 0.49,
            'cilantro': 1.99,
            'quinoa': 4.99,
            'chickpeas': 1.49,
            'sweet potato': 1.29,
            'kale': 2.99,
            'tahini': 7.99,
            'ground beef': 6.99,
            'burger buns': 3.99,
            'lettuce': 2.49,
            'tomato': 1.99,
            'cheddar cheese': 5.99,
            'mixed greens': 3.99,
            'cucumber': 1.49,
            'cherry tomatoes': 3.99,
            'feta cheese': 6.99,
            'olives': 4.99,
            'salmon fillets': 12.99,
            'teriyaki sauce': 4.99,
            'rice': 3.99,
            'broccoli': 2.99,
            'sesame seeds': 3.49,
            'eggs': 4.99,
            'milk': 3.99,
            'bread': 2.99,
            'butter': 4.99,
            'onion': 0.99,
            'garlic': 0.79,
            'carrots': 1.99,
            'potatoes': 2.99,
        }
        
        # Return price if found, otherwise estimate based on name length
        item_lower = item_name.lower()
        for key, price in prices.items():
            if key in item_lower:
                return price
        
        # Default fallback price
        return 3.99


## Travel Tool

In [ ]:
"""Travel Search MCP Tool for trip planning."""
import logging
import time
from typing import List, Dict, Optional
from datetime import datetime, timedelta

logger = logging.getLogger(__name__)


class TravelSearchTool:
    """MCP tool interface for accessing travel search APIs.
    
    This is a stub implementation with mock data for development.
    In production, this would connect to real travel APIs via MCP protocol.
    """
    
    def __init__(self, api_key: Optional[str] = None, max_retries: int = 3):
        """Initialize the travel search tool.
        
        Args:
            api_key: API key for travel service (optional for mock)
            max_retries: Maximum number of retry attempts for failed requests
        """
        self.api_key = api_key
        self.max_retries = max_retries
        self.call_count = 0  # For testing tool invocation
        logger.info("TravelSearchTool initialized")
    
    async def search_accommodations(
        self,
        destination: str,
        check_in: str,
        check_out: str,
        guests: int = 2,
        max_budget: Optional[float] = None,
        accommodation_type: Optional[str] = None
    ) -> List[Dict]:
        """Search for accommodations at destination.
        
        Args:
            destination: Destination city or location
            check_in: Check-in date (YYYY-MM-DD)
            check_out: Check-out date (YYYY-MM-DD)
            guests: Number of guests
            max_budget: Maximum budget per night
            accommodation_type: Type filter (hotel, airbnb, hostel, etc.)
            
        Returns:
            List of accommodation dictionaries
            
        Raises:
            Exception: If API call fails after retries
        """
        self.call_count += 1
        logger.info(f"Searching accommodations in {destination} for {guests} guests")
        
        # Simulate API call with retry logic
        for attempt in range(self.max_retries):
            try:
                accommodations = self._get_mock_accommodations(destination)
                
                # Filter by type
                if accommodation_type:
                    accommodations = [a for a in accommodations if a['type'].lower() == accommodation_type.lower()]
                
                # Filter by budget
                if max_budget:
                    accommodations = [a for a in accommodations if a['cost_per_night'] <= max_budget]
                
                # Calculate total cost
                check_in_date = datetime.strptime(check_in, '%Y-%m-%d')
                check_out_date = datetime.strptime(check_out, '%Y-%m-%d')
                nights = (check_out_date - check_in_date).days
                
                for acc in accommodations:
                    acc['nights'] = nights
                    acc['total_cost'] = acc['cost_per_night'] * nights
                    acc['check_in'] = check_in
                    acc['check_out'] = check_out
                
                return accommodations
                
            except Exception as e:
                logger.warning(f"Accommodation search attempt {attempt + 1} failed: {e}")
                if attempt < self.max_retries - 1:
                    wait_time = 2 ** attempt
                    logger.info(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    logger.error("Accommodation search failed after all retries")
                    raise
    
    async def search_activities(
        self,
        destination: str,
        interests: Optional[List[str]] = None,
        max_budget: Optional[float] = None
    ) -> List[Dict]:
        """Search for activities at destination.
        
        Args:
            destination: Destination city or location
            interests: List of interest categories (e.g., ["museums", "outdoor"])
            max_budget: Maximum budget per activity
            
        Returns:
            List of activity dictionaries
        """
        self.call_count += 1
        logger.info(f"Searching activities in {destination}")
        
        activities = self._get_mock_activities(destination)
        
        # Filter by interests
        if interests:
            activities = [a for a in activities if any(i.lower() in a['category'].lower() for i in interests)]
        
        # Filter by budget
        if max_budget:
            activities = [a for a in activities if a['estimated_cost'] <= max_budget]
        
        return activities
    
    async def search_restaurants(
        self,
        destination: str,
        cuisine_type: Optional[str] = None,
        price_range: Optional[str] = None
    ) -> List[Dict]:
        """Search for restaurants at destination.
        
        Args:
            destination: Destination city or location
            cuisine_type: Type of cuisine (e.g., "italian", "japanese")
            price_range: Price range ($, $$, $$$, $$$$)
            
        Returns:
            List of restaurant dictionaries
        """
        self.call_count += 1
        logger.info(f"Searching restaurants in {destination}")
        
        restaurants = self._get_mock_restaurants(destination)
        
        # Filter by cuisine
        if cuisine_type:
            restaurants = [r for r in restaurants if r['cuisine_type'].lower() == cuisine_type.lower()]
        
        # Filter by price range
        if price_range:
            restaurants = [r for r in restaurants if r['price_range'] == price_range]
        
        return restaurants
    
    async def get_destination_info(self, destination: str) -> Dict:
        """Get general information about a destination.
        
        Args:
            destination: Destination city or location
            
        Returns:
            Dictionary with destination information
        """
        self.call_count += 1
        logger.info(f"Getting info for destination: {destination}")
        
        # Mock destination data
        destinations = {
            'paris': {
                'name': 'Paris',
                'country': 'France',
                'description': 'The City of Light, known for art, fashion, and culture',
                'best_time_to_visit': 'April to June, September to October',
                'currency': 'EUR',
                'language': 'French',
                'timezone': 'CET',
                'popular_areas': ['Eiffel Tower', 'Louvre Museum', 'Notre-Dame', 'Champs-Élysées']
            },
            'tokyo': {
                'name': 'Tokyo',
                'country': 'Japan',
                'description': 'A vibrant metropolis blending tradition and modernity',
                'best_time_to_visit': 'March to May, September to November',
                'currency': 'JPY',
                'language': 'Japanese',
                'timezone': 'JST',
                'popular_areas': ['Shibuya', 'Shinjuku', 'Asakusa', 'Akihabara']
            },
            'new york': {
                'name': 'New York City',
                'country': 'USA',
                'description': 'The city that never sleeps, a global hub of culture and commerce',
                'best_time_to_visit': 'April to June, September to November',
                'currency': 'USD',
                'language': 'English',
                'timezone': 'EST',
                'popular_areas': ['Times Square', 'Central Park', 'Statue of Liberty', 'Brooklyn Bridge']
            }
        }
        
        dest_lower = destination.lower()
        for key, info in destinations.items():
            if key in dest_lower:
                return info
        
        # Default fallback
        return {
            'name': destination,
            'country': 'Unknown',
            'description': f'A wonderful destination: {destination}',
            'best_time_to_visit': 'Year-round',
            'currency': 'USD',
            'language': 'English',
            'timezone': 'UTC',
            'popular_areas': []
        }
    
    def _get_mock_accommodations(self, destination: str) -> List[Dict]:
        """Get mock accommodation data.
        
        Args:
            destination: Destination name
            
        Returns:
            List of mock accommodations
        """
        return [
            {
                'id': 'acc_001',
                'name': f'Grand Hotel {destination}',
                'type': 'hotel',
                'location': f'Downtown {destination}',
                'cost_per_night': 150.0,
                'rating': 4.5,
                'amenities': ['wifi', 'breakfast', 'gym', 'pool'],
                'description': 'Luxury hotel in the heart of the city'
            },
            {
                'id': 'acc_002',
                'name': f'Cozy Apartment {destination}',
                'type': 'airbnb',
                'location': f'City Center {destination}',
                'cost_per_night': 80.0,
                'rating': 4.7,
                'amenities': ['wifi', 'kitchen', 'washer'],
                'description': 'Modern apartment with great views'
            },
            {
                'id': 'acc_003',
                'name': f'Budget Hostel {destination}',
                'type': 'hostel',
                'location': f'Near Station {destination}',
                'cost_per_night': 35.0,
                'rating': 4.2,
                'amenities': ['wifi', 'shared kitchen', 'lounge'],
                'description': 'Affordable accommodation for backpackers'
            },
            {
                'id': 'acc_004',
                'name': f'Boutique Inn {destination}',
                'type': 'hotel',
                'location': f'Historic District {destination}',
                'cost_per_night': 120.0,
                'rating': 4.8,
                'amenities': ['wifi', 'breakfast', 'spa'],
                'description': 'Charming boutique hotel with personalized service'
            }
        ]
    
    def _get_mock_activities(self, destination: str) -> List[Dict]:
        """Get mock activity data.
        
        Args:
            destination: Destination name
            
        Returns:
            List of mock activities
        """
        return [
            {
                'id': 'act_001',
                'name': f'{destination} City Tour',
                'category': 'sightseeing',
                'description': 'Guided tour of major landmarks',
                'duration': 180,  # minutes
                'estimated_cost': 45.0,
                'rating': 4.6
            },
            {
                'id': 'act_002',
                'name': f'{destination} Museum Visit',
                'category': 'museums',
                'description': 'Explore world-class art and history',
                'duration': 120,
                'estimated_cost': 25.0,
                'rating': 4.7
            },
            {
                'id': 'act_003',
                'name': f'{destination} Food Tour',
                'category': 'food',
                'description': 'Taste local cuisine and specialties',
                'duration': 150,
                'estimated_cost': 65.0,
                'rating': 4.9
            },
            {
                'id': 'act_004',
                'name': f'{destination} Park Walk',
                'category': 'outdoor',
                'description': 'Relaxing walk through scenic parks',
                'duration': 90,
                'estimated_cost': 0.0,
                'rating': 4.5
            },
            {
                'id': 'act_005',
                'name': f'{destination} Shopping District',
                'category': 'shopping',
                'description': 'Browse local shops and boutiques',
                'duration': 120,
                'estimated_cost': 0.0,
                'rating': 4.3
            }
        ]
    
    def _get_mock_restaurants(self, destination: str) -> List[Dict]:
        """Get mock restaurant data.
        
        Args:
            destination: Destination name
            
        Returns:
            List of mock restaurants
        """
        return [
            {
                'id': 'rest_001',
                'name': f'Le Bistro {destination}',
                'cuisine_type': 'French',
                'price_range': '$$$',
                'estimated_cost': 50.0,
                'rating': 4.6,
                'location': f'Downtown {destination}',
                'description': 'Classic French cuisine in elegant setting'
            },
            {
                'id': 'rest_002',
                'name': f'Sushi Bar {destination}',
                'cuisine_type': 'Japanese',
                'price_range': '$$',
                'estimated_cost': 35.0,
                'rating': 4.7,
                'location': f'City Center {destination}',
                'description': 'Fresh sushi and authentic Japanese dishes'
            },
            {
                'id': 'rest_003',
                'name': f'Pizza Place {destination}',
                'cuisine_type': 'Italian',
                'price_range': '$',
                'estimated_cost': 20.0,
                'rating': 4.4,
                'location': f'Near Station {destination}',
                'description': 'Casual Italian pizzeria with great value'
            },
            {
                'id': 'rest_004',
                'name': f'Steakhouse {destination}',
                'cuisine_type': 'American',
                'price_range': '$$$$',
                'estimated_cost': 75.0,
                'rating': 4.8,
                'location': f'Uptown {destination}',
                'description': 'Premium steaks and fine dining experience'
            }
        ]


## Meal Planning Agent

In [ ]:
"""Meal Planning Agent for generating meal plans and recipes."""
import logging
import uuid
from typing import List, Dict, Optional
from datetime import date, timedelta

from data_models import MealPlan, Meal, Ingredient
from recipe_tool import RecipeDatabaseTool

logger = logging.getLogger(__name__)


class MealPlanningAgent:
    """Agent responsible for creating meal plans based on user preferences.
    
    This agent generates meal plans for specified time periods, applies dietary
    restrictions and preferences, and provides detailed recipe information.
    """
    
    def __init__(self, recipe_tool: RecipeDatabaseTool):
        """Initialize the Meal Planning Agent.
        
        Args:
            recipe_tool: MCP tool for accessing recipe database
        """
        self.recipe_tool = recipe_tool
        logger.info("MealPlanningAgent initialized")
    
    async def generate_meal_plan(
        self,
        user_id: str,
        days: int,
        preferences: Optional[Dict] = None
    ) -> MealPlan:
        """Generate a meal plan for the specified number of days.
        
        Args:
            user_id: User identifier
            days: Number of days to plan for
            preferences: Dictionary containing:
                - dietary_restrictions: List of dietary restrictions
                - cuisine_preferences: List of preferred cuisines
                - meals_per_day: Number of meals per day (default: 3)
                
        Returns:
            MealPlan object with meals for the specified period
            
        Raises:
            ValueError: If days is less than 1
        """
        if days < 1:
            raise ValueError("Days must be at least 1")
        
        preferences = preferences or {}
        dietary_restrictions = preferences.get('dietary_restrictions', [])
        cuisine_preferences = preferences.get('cuisine_preferences', [])
        meals_per_day = preferences.get('meals_per_day', 3)
        
        logger.info(f"Generating {days}-day meal plan for user {user_id}")
        logger.info(f"Dietary restrictions: {dietary_restrictions}")
        logger.info(f"Cuisine preferences: {cuisine_preferences}")
        
        # Calculate date range
        start_date = date.today()
        end_date = start_date + timedelta(days=days - 1)
        
        # Generate meals
        meals = []
        meal_types = self._get_meal_types(meals_per_day)
        
        for day in range(days):
            current_date = start_date + timedelta(days=day)
            
            for meal_type in meal_types:
                # Search for recipes matching preferences
                cuisine = None
                if cuisine_preferences:
                    # Rotate through cuisine preferences
                    cuisine = cuisine_preferences[len(meals) % len(cuisine_preferences)]
                
                recipes = await self.recipe_tool.search_recipes(
                    cuisine=cuisine,
                    dietary_restrictions=dietary_restrictions,
                    max_results=5
                )
                
                if not recipes:
                    # Fallback: get random recipes
                    recipes = await self.recipe_tool.get_random_recipes(
                        count=5,
                        dietary_restrictions=dietary_restrictions
                    )
                
                if recipes:
                    # Select a recipe (simple selection for now)
                    recipe = recipes[0]
                    meal = self._recipe_to_meal(recipe, meal_type)
                    meals.append(meal)
        
        # Create meal plan
        plan_id = f"plan_{uuid.uuid4().hex[:8]}"
        unique_recipes = len(set(m.recipe_id for m in meals))
        
        meal_plan = MealPlan(
            plan_id=plan_id,
            user_id=user_id,
            start_date=start_date,
            end_date=end_date,
            meals=meals,
            total_recipes=unique_recipes
        )
        
        # Validate the meal plan
        meal_plan.validate()
        
        logger.info(f"Generated meal plan {plan_id} with {len(meals)} meals")
        return meal_plan
    
    async def get_recipe_details(self, recipe_id: str) -> Dict:
        """Get detailed information for a specific recipe.
        
        Args:
            recipe_id: Unique recipe identifier
            
        Returns:
            Dictionary with complete recipe details
            
        Raises:
            ValueError: If recipe not found
        """
        logger.info(f"Fetching details for recipe {recipe_id}")
        recipe = await self.recipe_tool.get_recipe_details(recipe_id)
        return recipe
    
    async def apply_dietary_restrictions(
        self,
        recipes: List[Dict],
        restrictions: List[str]
    ) -> List[Dict]:
        """Filter recipes to match dietary restrictions.
        
        Args:
            recipes: List of recipe dictionaries
            restrictions: List of dietary restrictions (e.g., ["vegetarian", "gluten-free"])
            
        Returns:
            Filtered list of recipes matching all restrictions
        """
        if not restrictions:
            return recipes
        
        logger.info(f"Applying dietary restrictions: {restrictions}")
        
        filtered = []
        for recipe in recipes:
            recipe_tags = [tag.lower() for tag in recipe.get('dietary_tags', [])]
            
            # Check if recipe matches all restrictions
            if all(restriction.lower() in recipe_tags for restriction in restrictions):
                filtered.append(recipe)
        
        logger.info(f"Filtered {len(recipes)} recipes to {len(filtered)} matching restrictions")
        return filtered
    
    async def filter_by_cuisine(
        self,
        recipes: List[Dict],
        cuisine: str
    ) -> List[Dict]:
        """Filter recipes by cuisine type.
        
        Args:
            recipes: List of recipe dictionaries
            cuisine: Cuisine type to filter by
            
        Returns:
            Filtered list of recipes matching cuisine
        """
        logger.info(f"Filtering recipes by cuisine: {cuisine}")
        
        filtered = [r for r in recipes if r['cuisine'].lower() == cuisine.lower()]
        
        logger.info(f"Filtered {len(recipes)} recipes to {len(filtered)} matching cuisine")
        return filtered
    
    def _get_meal_types(self, meals_per_day: int) -> List[str]:
        """Get meal types based on meals per day.
        
        Args:
            meals_per_day: Number of meals per day
            
        Returns:
            List of meal type strings
        """
        if meals_per_day == 1:
            return ['dinner']
        elif meals_per_day == 2:
            return ['lunch', 'dinner']
        elif meals_per_day == 3:
            return ['breakfast', 'lunch', 'dinner']
        else:
            # 4+ meals: add snacks
            return ['breakfast', 'lunch', 'snack', 'dinner']
    
    def _recipe_to_meal(self, recipe: Dict, meal_type: str) -> Meal:
        """Convert a recipe dictionary to a Meal object.
        
        Args:
            recipe: Recipe dictionary from recipe tool
            meal_type: Type of meal (breakfast, lunch, dinner, snack)
            
        Returns:
            Meal object
        """
        # Convert ingredients
        ingredients = []
        for ing in recipe.get('ingredients', []):
            ingredient = Ingredient(
                name=ing['name'],
                quantity=ing['quantity'],
                unit=ing['unit']
            )
            ingredients.append(ingredient)
        
        # Create meal
        meal = Meal(
            meal_type=meal_type,
            recipe_id=recipe['id'],
            recipe_name=recipe['name'],
            ingredients=ingredients,
            instructions=recipe.get('instructions', ''),
            prep_time=recipe.get('prep_time', 0),
            cook_time=recipe.get('cook_time', 0)
        )
        
        return meal


## Shopping Agent

In [ ]:
"""Shopping Agent for generating and managing shopping lists."""
import logging
import uuid
from typing import List, Dict, Optional
from datetime import date
from collections import defaultdict

from data_models import ShoppingList, ShoppingItem, MealPlan, Ingredient
from pricing_tool import PricingAPITool

logger = logging.getLogger(__name__)


class ShoppingAgent:
    """Agent responsible for generating shopping lists from meal plans.
    
    This agent extracts ingredients, consolidates quantities, organizes by
    category, and integrates with pricing APIs.
    """
    
    # Category mapping for common ingredients
    CATEGORY_MAP = {
        'pasta': 'Grains & Pasta',
        'rice': 'Grains & Pasta',
        'quinoa': 'Grains & Pasta',
        'bread': 'Bakery',
        'burger buns': 'Bakery',
        'corn tortillas': 'Bakery',
        'chicken breast': 'Meat & Poultry',
        'ground beef': 'Meat & Poultry',
        'salmon fillets': 'Seafood',
        'eggs': 'Dairy & Eggs',
        'milk': 'Dairy & Eggs',
        'butter': 'Dairy & Eggs',
        'cheddar cheese': 'Dairy & Eggs',
        'parmesan cheese': 'Dairy & Eggs',
        'feta cheese': 'Dairy & Eggs',
        'bell peppers': 'Produce',
        'zucchini': 'Produce',
        'avocado': 'Produce',
        'lime': 'Produce',
        'cilantro': 'Produce',
        'sweet potato': 'Produce',
        'kale': 'Produce',
        'lettuce': 'Produce',
        'tomato': 'Produce',
        'cucumber': 'Produce',
        'cherry tomatoes': 'Produce',
        'mixed greens': 'Produce',
        'broccoli': 'Produce',
        'onion': 'Produce',
        'garlic': 'Produce',
        'carrots': 'Produce',
        'potatoes': 'Produce',
        'chickpeas': 'Canned Goods',
        'olives': 'Canned Goods',
        'olive oil': 'Oils & Condiments',
        'teriyaki sauce': 'Oils & Condiments',
        'tahini': 'Oils & Condiments',
        'sesame seeds': 'Spices & Seasonings',
    }
    
    def __init__(self, pricing_tool: PricingAPITool):
        """Initialize the Shopping Agent.
        
        Args:
            pricing_tool: MCP tool for accessing pricing information
        """
        self.pricing_tool = pricing_tool
        logger.info("ShoppingAgent initialized")
    
    async def generate_shopping_list(
        self,
        user_id: str,
        meal_plan: MealPlan,
        pantry: Optional[List[str]] = None
    ) -> ShoppingList:
        """Generate a shopping list from a meal plan.
        
        Args:
            user_id: User identifier
            meal_plan: MealPlan object containing meals
            pantry: List of items already in pantry (to exclude)
            
        Returns:
            ShoppingList object with consolidated and categorized items
        """
        pantry = pantry or []
        logger.info(f"Generating shopping list for meal plan {meal_plan.plan_id}")
        logger.info(f"Pantry items to exclude: {pantry}")
        
        # Extract all ingredients from meals
        all_ingredients = []
        for meal in meal_plan.meals:
            all_ingredients.extend(meal.ingredients)
        
        logger.info(f"Extracted {len(all_ingredients)} ingredients from {len(meal_plan.meals)} meals")
        
        # Consolidate duplicate ingredients
        consolidated = await self.consolidate_ingredients(all_ingredients)
        
        # Filter out pantry items
        filtered = [ing for ing in consolidated if ing['name'].lower() not in [p.lower() for p in pantry]]
        logger.info(f"Filtered to {len(filtered)} items after removing pantry items")
        
        # Get price estimates
        prices = await self.get_price_estimates(filtered)
        
        # Create shopping items with categories
        shopping_items = []
        for item_data in filtered:
            price_info = next((p for p in prices if p['item_name'] == item_data['name']), None)
            estimated_price = price_info['total_price'] if price_info else 0.0
            
            category = self._categorize_item(item_data['name'])
            
            shopping_item = ShoppingItem(
                name=item_data['name'],
                quantity=item_data['quantity'],
                unit=item_data['unit'],
                category=category,
                estimated_price=estimated_price
            )
            shopping_items.append(shopping_item)
        
        # Organize by category
        categories = await self.organize_by_category(shopping_items)
        
        # Calculate total
        estimated_total = sum(item.estimated_price for item in shopping_items)
        
        # Create shopping list
        list_id = f"list_{uuid.uuid4().hex[:8]}"
        shopping_list = ShoppingList(
            list_id=list_id,
            user_id=user_id,
            created_date=date.today(),
            items=shopping_items,
            categories=categories,
            estimated_total=round(estimated_total, 2)
        )
        
        # Validate
        shopping_list.validate()
        
        logger.info(f"Generated shopping list {list_id} with {len(shopping_items)} items, total: ${estimated_total:.2f}")
        return shopping_list
    
    async def consolidate_ingredients(
        self,
        ingredients: List[Ingredient]
    ) -> List[Dict]:
        """Consolidate duplicate ingredients with correct quantity summing.
        
        Args:
            ingredients: List of Ingredient objects
            
        Returns:
            List of consolidated ingredient dictionaries
        """
        logger.info(f"Consolidating {len(ingredients)} ingredients")
        
        # Group by name and unit
        grouped = defaultdict(lambda: {'quantity': 0.0, 'unit': '', 'name': ''})
        
        for ing in ingredients:
            key = (ing.name.lower(), ing.unit.lower())
            grouped[key]['name'] = ing.name
            grouped[key]['unit'] = ing.unit
            grouped[key]['quantity'] += ing.quantity
        
        # Convert to list
        consolidated = [
            {
                'name': data['name'],
                'quantity': round(data['quantity'], 2),
                'unit': data['unit']
            }
            for data in grouped.values()
        ]
        
        logger.info(f"Consolidated to {len(consolidated)} unique items")
        return consolidated
    
    async def organize_by_category(
        self,
        items: List[ShoppingItem]
    ) -> Dict[str, List[ShoppingItem]]:
        """Organize shopping items by grocery store category.
        
        Args:
            items: List of ShoppingItem objects
            
        Returns:
            Dictionary mapping category names to lists of items
        """
        logger.info(f"Organizing {len(items)} items by category")
        
        categories = defaultdict(list)
        for item in items:
            categories[item.category].append(item)
        
        # Sort items within each category by name
        for category in categories:
            categories[category].sort(key=lambda x: x.name)
        
        logger.info(f"Organized into {len(categories)} categories")
        return dict(categories)
    
    async def get_price_estimates(
        self,
        items: List[Dict]
    ) -> List[Dict]:
        """Get price estimates for shopping items.
        
        Args:
            items: List of item dictionaries with name, quantity, unit
            
        Returns:
            List of price information dictionaries
        """
        logger.info(f"Getting price estimates for {len(items)} items")
        
        prices = await self.pricing_tool.get_bulk_prices(items)
        
        return prices
    
    def _categorize_item(self, item_name: str) -> str:
        """Categorize an item by name.
        
        Args:
            item_name: Name of the item
            
        Returns:
            Category name
        """
        item_lower = item_name.lower()
        
        # Check exact matches first
        if item_lower in self.CATEGORY_MAP:
            return self.CATEGORY_MAP[item_lower]
        
        # Check partial matches
        for key, category in self.CATEGORY_MAP.items():
            if key in item_lower or item_lower in key:
                return category
        
        # Default category
        return 'Other'


## Travel Agent

In [ ]:
"""Travel Agent for planning trips and creating itineraries."""
import logging
import uuid
from typing import List, Dict, Optional, Tuple
from datetime import date, timedelta

from data_models import (
    TripPlan, DayPlan, Activity, Restaurant, Accommodation
)
from travel_tool import TravelSearchTool

logger = logging.getLogger(__name__)


class TravelAgent:
    """Agent responsible for planning trips and creating itineraries.
    
    This agent gathers trip requirements, searches accommodations, creates
    day-by-day itineraries, and recommends activities and restaurants.
    """
    
    def __init__(self, travel_tool: TravelSearchTool):
        """Initialize the Travel Agent.
        
        Args:
            travel_tool: MCP tool for accessing travel search APIs
        """
        self.travel_tool = travel_tool
        logger.info("TravelAgent initialized")
    
    async def plan_trip(
        self,
        user_id: str,
        destination: str,
        dates: Tuple[str, str],
        budget: float,
        preferences: Optional[Dict] = None
    ) -> TripPlan:
        """Plan a complete trip with accommodation and itinerary.
        
        Args:
            user_id: User identifier
            destination: Destination city or location
            dates: Tuple of (start_date, end_date) in YYYY-MM-DD format
            budget: Total budget for the trip
            preferences: Dictionary containing:
                - accommodation_type: Preferred type (hotel, airbnb, hostel)
                - interests: List of interests (museums, outdoor, food, etc.)
                - budget_per_night: Maximum budget per night for accommodation
                
        Returns:
            TripPlan object with accommodation and daily itinerary
            
        Raises:
            ValueError: If dates are invalid or budget is negative
        """
        if budget < 0:
            raise ValueError("Budget must be non-negative")
        
        start_date_str, end_date_str = dates
        start_date = date.fromisoformat(start_date_str)
        end_date = date.fromisoformat(end_date_str)
        
        if start_date > end_date:
            raise ValueError("Start date must be before or equal to end date")
        
        preferences = preferences or {}
        accommodation_type = preferences.get('accommodation_type')
        interests = preferences.get('interests', [])
        budget_per_night = preferences.get('budget_per_night')
        
        logger.info(f"Planning trip to {destination} from {start_date} to {end_date}")
        logger.info(f"Budget: ${budget}, Interests: {interests}")
        
        # Calculate trip duration
        nights = (end_date - start_date).days
        days = nights + 1
        
        # Search for accommodations
        accommodations = await self.search_accommodations(
            destination=destination,
            check_in=start_date_str,
            check_out=end_date_str,
            max_budget=budget_per_night,
            accommodation_type=accommodation_type
        )
        
        if not accommodations:
            raise ValueError(f"No accommodations found in {destination} within budget")
        
        # Select best accommodation (first one, sorted by rating in tool)
        selected_accommodation = accommodations[0]
        
        # Create itinerary
        itinerary = await self.create_itinerary(
            destination=destination,
            start_date=start_date,
            days=days,
            interests=interests,
            daily_budget=(budget - selected_accommodation['total_cost']) / days if days > 0 else 0
        )
        
        # Convert accommodation dict to Accommodation object
        accommodation_obj = Accommodation(
            name=selected_accommodation['name'],
            type=selected_accommodation['type'],
            location=selected_accommodation['location'],
            cost_per_night=selected_accommodation['cost_per_night'],
            total_cost=selected_accommodation['total_cost'],
            amenities=selected_accommodation.get('amenities', [])
        )
        
        # Calculate total estimated cost
        accommodation_cost = selected_accommodation['total_cost']
        activities_cost = sum(
            sum(a.estimated_cost for a in day.activities)
            for day in itinerary
        )
        meals_cost = sum(
            sum(r.estimated_cost for r in day.meals)
            for day in itinerary
        )
        estimated_cost = accommodation_cost + activities_cost + meals_cost
        
        # Create trip plan
        trip_id = f"trip_{uuid.uuid4().hex[:8]}"
        trip_plan = TripPlan(
            trip_id=trip_id,
            user_id=user_id,
            destination=destination,
            start_date=start_date,
            end_date=end_date,
            accommodation=accommodation_obj,
            itinerary=itinerary,
            estimated_cost=round(estimated_cost, 2)
        )
        
        # Validate
        trip_plan.validate()
        
        logger.info(f"Created trip plan {trip_id} for {days} days, estimated cost: ${estimated_cost:.2f}")
        return trip_plan
    
    async def search_accommodations(
        self,
        destination: str,
        check_in: str,
        check_out: str,
        max_budget: Optional[float] = None,
        accommodation_type: Optional[str] = None
    ) -> List[Dict]:
        """Search for accommodations at destination.
        
        Args:
            destination: Destination city or location
            check_in: Check-in date (YYYY-MM-DD)
            check_out: Check-out date (YYYY-MM-DD)
            max_budget: Maximum budget per night
            accommodation_type: Type filter (hotel, airbnb, hostel)
            
        Returns:
            List of accommodation dictionaries
        """
        logger.info(f"Searching accommodations in {destination}")
        
        accommodations = await self.travel_tool.search_accommodations(
            destination=destination,
            check_in=check_in,
            check_out=check_out,
            guests=2,
            max_budget=max_budget,
            accommodation_type=accommodation_type
        )
        
        # Filter by budget if specified
        if max_budget:
            accommodations = [a for a in accommodations if a['cost_per_night'] <= max_budget]
        
        logger.info(f"Found {len(accommodations)} accommodations")
        return accommodations
    
    async def create_itinerary(
        self,
        destination: str,
        start_date: date,
        days: int,
        interests: Optional[List[str]] = None,
        daily_budget: float = 100.0
    ) -> List[DayPlan]:
        """Create a day-by-day itinerary for the trip.
        
        Args:
            destination: Destination city or location
            start_date: Start date of the trip
            days: Number of days
            interests: List of interest categories
            daily_budget: Budget per day for activities and meals
            
        Returns:
            List of DayPlan objects
        """
        logger.info(f"Creating {days}-day itinerary for {destination}")
        
        interests = interests or []
        
        # Get activities and restaurants
        activities = await self.suggest_activities(destination, interests)
        restaurants = await self.travel_tool.search_restaurants(destination)
        
        itinerary = []
        
        for day_num in range(1, days + 1):
            current_date = start_date + timedelta(days=day_num - 1)
            
            # Select 2-3 activities per day
            day_activities = []
            activities_per_day = min(3, len(activities))
            
            for i in range(activities_per_day):
                # Rotate through available activities
                activity_data = activities[(day_num - 1 + i) % len(activities)]
                
                activity = Activity(
                    name=activity_data['name'],
                    description=activity_data['description'],
                    duration=activity_data['duration'],
                    estimated_cost=activity_data['estimated_cost'],
                    location=activity_data.get('location', destination)
                )
                day_activities.append(activity)
            
            # Select 2-3 restaurants per day (breakfast, lunch, dinner)
            day_restaurants = []
            meals_per_day = min(3, len(restaurants))
            
            for i in range(meals_per_day):
                # Rotate through available restaurants
                restaurant_data = restaurants[(day_num - 1 + i) % len(restaurants)]
                
                restaurant = Restaurant(
                    name=restaurant_data['name'],
                    cuisine_type=restaurant_data['cuisine_type'],
                    estimated_cost=restaurant_data['estimated_cost'],
                    location=restaurant_data.get('location', destination)
                )
                day_restaurants.append(restaurant)
            
            # Create day plan
            day_plan = DayPlan(
                day_number=day_num,
                date=current_date,
                activities=day_activities,
                meals=day_restaurants,
                notes=f"Day {day_num} in {destination}"
            )
            
            itinerary.append(day_plan)
        
        logger.info(f"Created itinerary with {len(itinerary)} days")
        return itinerary
    
    async def suggest_activities(
        self,
        destination: str,
        interests: List[str]
    ) -> List[Dict]:
        """Suggest activities based on interests.
        
        Args:
            destination: Destination city or location
            interests: List of interest categories
            
        Returns:
            List of activity dictionaries
        """
        logger.info(f"Suggesting activities for {destination} with interests: {interests}")
        
        activities = await self.travel_tool.search_activities(
            destination=destination,
            interests=interests if interests else None
        )
        
        logger.info(f"Found {len(activities)} activities")
        return activities


## Orchestrator Agent

In [ ]:
"""Orchestrator Agent for coordinating specialized agents."""
import logging
import re
from typing import Dict, Optional, Any
from datetime import date, timedelta

from meal_planning_agent import MealPlanningAgent
from shopping_agent import ShoppingAgent
from travel_agent import TravelAgent
from session_manager import SessionManager
from memory_bank import MemoryBank

logger = logging.getLogger(__name__)


class OrchestratorAgent:
    """Main orchestrator agent that coordinates specialized sub-agents.
    
    This agent parses user intent, routes to appropriate sub-agents,
    maintains conversation context, and manages session lifecycle.
    """
    
    def __init__(
        self,
        meal_agent: MealPlanningAgent,
        shopping_agent: ShoppingAgent,
        travel_agent: TravelAgent,
        session_manager: SessionManager,
        memory_bank: MemoryBank
    ):
        """Initialize the Orchestrator Agent.
        
        Args:
            meal_agent: Meal planning agent instance
            shopping_agent: Shopping agent instance
            travel_agent: Travel agent instance
            session_manager: Session manager for conversation state
            memory_bank: Memory bank for user preferences
        """
        self.meal_agent = meal_agent
        self.shopping_agent = shopping_agent
        self.travel_agent = travel_agent
        self.session_manager = session_manager
        self.memory_bank = memory_bank
        logger.info("OrchestratorAgent initialized")
    
    async def process_message(
        self,
        user_id: str,
        message: str,
        session_id: Optional[str] = None
    ) -> Dict[str, Any]:
        """Process a user message and route to appropriate agent.
        
        Args:
            user_id: User identifier
            message: User message text
            session_id: Optional session ID (creates new if not provided)
            
        Returns:
            Dictionary with response, intent, and any generated data
        """
        logger.info(f"Processing message from user {user_id}: {message[:50]}...")
        
        # Get or create session
        if not session_id:
            session_id = await self.session_manager.create_session(user_id)
            logger.info(f"Created new session: {session_id}")
        
        # Load user preferences
        preferences = await self.memory_bank.get_all_preferences(user_id)
        
        # Parse intent
        intent = self._parse_intent(message)
        logger.info(f"Detected intent: {intent}")
        
        # Check for ambiguity
        if intent == 'ambiguous':
            return {
                'response': self._generate_clarifying_questions(message),
                'intent': 'ambiguous',
                'requires_clarification': True
            }
        
        # Route to appropriate agent
        context = {
            'user_id': user_id,
            'session_id': session_id,
            'preferences': preferences,
            'message': message
        }
        
        result = await self.route_to_agent(intent, context)
        
        # Generate summary
        summary = self._generate_summary(intent, result)
        
        # Update session
        await self.session_manager.update_session(
            session_id,
            [{'role': 'user', 'content': message}, {'role': 'assistant', 'content': summary}]
        )
        
        return {
            'response': summary,
            'intent': intent,
            'data': result.get('data'),
            'session_id': session_id
        }
    
    async def route_to_agent(
        self,
        intent: str,
        context: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Route request to appropriate specialized agent.
        
        Args:
            intent: Detected intent (meal_planning, shopping, travel, etc.)
            context: Context dictionary with user info and preferences
            
        Returns:
            Dictionary with agent response and generated data
        """
        logger.info(f"Routing to agent for intent: {intent}")
        
        user_id = context['user_id']
        preferences = context.get('preferences', {})
        message = context.get('message', '')
        
        try:
            if intent == 'meal_planning':
                # Extract parameters from message
                days = self._extract_days(message) or 7
                
                meal_plan = await self.meal_agent.generate_meal_plan(
                    user_id=user_id,
                    days=days,
                    preferences=preferences
                )
                
                return {
                    'success': True,
                    'data': meal_plan,
                    'message': f'Generated {days}-day meal plan'
                }
            
            elif intent == 'shopping':
                # For shopping, we need a meal plan
                # In a real implementation, this would retrieve from context
                return {
                    'success': False,
                    'message': 'Please generate a meal plan first before creating a shopping list'
                }
            
            elif intent == 'travel':
                # Extract travel parameters
                destination = self._extract_destination(message)
                if not destination:
                    return {
                        'success': False,
                        'message': 'Please specify a destination for your trip'
                    }
                
                # Use default dates if not specified
                start_date = date.today() + timedelta(days=30)
                end_date = start_date + timedelta(days=7)
                budget = 2000.0
                
                trip_plan = await self.travel_agent.plan_trip(
                    user_id=user_id,
                    destination=destination,
                    dates=(start_date.isoformat(), end_date.isoformat()),
                    budget=budget,
                    preferences=preferences
                )
                
                return {
                    'success': True,
                    'data': trip_plan,
                    'message': f'Created trip plan for {destination}'
                }
            
            elif intent == 'multi_domain':
                # Handle multi-domain requests sequentially
                results = []
                
                if 'meal' in message.lower():
                    meal_result = await self.route_to_agent('meal_planning', context)
                    results.append(meal_result)
                
                if 'shop' in message.lower():
                    shop_result = await self.route_to_agent('shopping', context)
                    results.append(shop_result)
                
                if 'travel' in message.lower() or 'trip' in message.lower():
                    travel_result = await self.route_to_agent('travel', context)
                    results.append(travel_result)
                
                return {
                    'success': True,
                    'data': results,
                    'message': 'Completed multi-domain request'
                }
            
            else:
                return {
                    'success': False,
                    'message': f'Unknown intent: {intent}'
                }
        
        except Exception as e:
            logger.error(f"Error routing to agent: {e}", exc_info=True)
            return {
                'success': False,
                'message': f'Error processing request: {str(e)}'
            }
    
    def _parse_intent(self, message: str) -> str:
        """Parse user intent from message.
        
        Args:
            message: User message text
            
        Returns:
            Intent string (meal_planning, shopping, travel, multi_domain, ambiguous)
        """
        message_lower = message.lower()
        
        # Count domain indicators
        domains = []
        
        if any(word in message_lower for word in ['meal', 'recipe', 'cook', 'eat', 'food', 'dinner', 'lunch', 'breakfast']):
            domains.append('meal_planning')
        
        if any(word in message_lower for word in ['shop', 'grocery', 'groceries', 'buy', 'ingredient', 'store']):
            domains.append('shopping')
        
        if any(word in message_lower for word in ['travel', 'trip', 'vacation', 'visit', 'hotel', 'flight']):
            domains.append('travel')
        
        # Determine intent
        if len(domains) == 0:
            return 'ambiguous'
        elif len(domains) == 1:
            return domains[0]
        else:
            return 'multi_domain'
    
    def _generate_clarifying_questions(self, message: str) -> str:
        """Generate clarifying questions for ambiguous input.
        
        Args:
            message: User message text
            
        Returns:
            Clarifying question string
        """
        return (
            "I'm not sure what you'd like help with. "
            "I can assist with:\n"
            "- Meal planning (creating weekly meal plans)\n"
            "- Shopping lists (generating grocery lists)\n"
            "- Travel planning (planning trips and itineraries)\n\n"
            "What would you like to do?"
        )
    
    def _generate_summary(self, intent: str, result: Dict[str, Any]) -> str:
        """Generate a summary of actions taken.
        
        Args:
            intent: Detected intent
            result: Result from agent execution
            
        Returns:
            Summary string
        """
        if not result.get('success'):
            return result.get('message', 'Failed to process request')
        
        if intent == 'meal_planning':
            meal_plan = result.get('data')
            if meal_plan:
                return (
                    f"✓ Created a {meal_plan.duration_days()}-day meal plan with "
                    f"{len(meal_plan.meals)} meals using {meal_plan.total_recipes} recipes."
                )
        
        elif intent == 'shopping':
            shopping_list = result.get('data')
            if shopping_list:
                return (
                    f"✓ Generated shopping list with {len(shopping_list.items)} items "
                    f"across {len(shopping_list.categories)} categories. "
                    f"Estimated total: ${shopping_list.estimated_total:.2f}"
                )
        
        elif intent == 'travel':
            trip_plan = result.get('data')
            if trip_plan:
                return (
                    f"✓ Planned {trip_plan.duration_days()}-day trip to {trip_plan.destination}. "
                    f"Accommodation: {trip_plan.accommodation.name}. "
                    f"Estimated cost: ${trip_plan.estimated_cost:.2f}"
                )
        
        elif intent == 'multi_domain':
            results = result.get('data', [])
            summaries = []
            for r in results:
                if r.get('success'):
                    summaries.append(r.get('message', ''))
            return "✓ Completed multiple tasks:\n" + "\n".join(f"  - {s}" for s in summaries)
        
        return result.get('message', 'Task completed')
    
    def _extract_days(self, message: str) -> Optional[int]:
        """Extract number of days from message.
        
        Args:
            message: User message text
            
        Returns:
            Number of days or None
        """
        # Look for patterns like "3 days", "5-day", "week" (7 days)
        patterns = [
            r'(\d+)\s*days?',
            r'(\d+)-day',
            r'for\s+(\d+)\s+days?'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, message.lower())
            if match:
                return int(match.group(1))
        
        if 'week' in message.lower():
            return 7
        
        return None
    
    def _extract_destination(self, message: str) -> Optional[str]:
        """Extract destination from message.
        
        Args:
            message: User message text
            
        Returns:
            Destination string or None
        """
        # Look for patterns like "to Paris", "visit Tokyo", "trip to New York"
        patterns = [
            r'to\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)',
            r'visit\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)',
            r'trip\s+to\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)',
            r'in\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, message)
            if match:
                return match.group(1)
        
        return None


## Gradio Interface

In [ ]:
"""Gradio web interface for the Personal Life Automation Agent System."""
import gradio as gr
import asyncio
import logging
from datetime import date, timedelta
import json

from meal_planning_agent import MealPlanningAgent
from shopping_agent import ShoppingAgent
from travel_agent import TravelAgent
from orchestrator_agent import OrchestratorAgent
from memory_bank import MemoryBank
from session_manager import SessionManager, InMemorySessionService
from recipe_tool import RecipeDatabaseTool
from pricing_tool import PricingAPITool
from travel_tool import TravelSearchTool

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Initialize all components
recipe_tool = RecipeDatabaseTool()
pricing_tool = PricingAPITool()
travel_tool = TravelSearchTool()

meal_agent = MealPlanningAgent(recipe_tool)
shopping_agent = ShoppingAgent(pricing_tool)
travel_agent = TravelAgent(travel_tool)

memory_bank = MemoryBank("app_memory.json")
session_service = InMemorySessionService()
session_manager = SessionManager(session_service, memory_bank)

orchestrator = OrchestratorAgent(
    meal_agent=meal_agent,
    shopping_agent=shopping_agent,
    travel_agent=travel_agent,
    session_manager=session_manager,
    memory_bank=memory_bank
)

# Store session IDs per user
user_sessions = {}


async def chat_with_agent(message, user_id="default_user"):
    """Process a chat message with the orchestrator agent."""
    try:
        # Get or create session for user
        session_id = user_sessions.get(user_id)
        
        response = await orchestrator.process_message(
            user_id=user_id,
            message=message,
            session_id=session_id
        )
        
        # Store session ID
        user_sessions[user_id] = response['session_id']
        
        return response['response']
    except Exception as e:
        return f"Error: {str(e)}"


async def generate_meal_plan(days, dietary_restrictions, cuisine_preferences, meals_per_day):
    """Generate a meal plan."""
    try:
        restrictions = [r.strip() for r in dietary_restrictions.split(',')] if dietary_restrictions else []
        cuisines = [c.strip() for c in cuisine_preferences.split(',')] if cuisine_preferences else []
        
        preferences = {
            'dietary_restrictions': restrictions,
            'cuisine_preferences': cuisines,
            'meals_per_day': int(meals_per_day)
        }
        
        meal_plan = await meal_agent.generate_meal_plan(
            user_id='gradio_user',
            days=int(days),
            preferences=preferences
        )
        
        # Format output
        output = f"# Meal Plan: {meal_plan.plan_id}\n\n"
        output += f"**Duration:** {meal_plan.duration_days()} days\n"
        output += f"**Total Meals:** {len(meal_plan.meals)}\n"
        output += f"**Unique Recipes:** {meal_plan.total_recipes}\n\n"
        output += "## Meals:\n\n"
        
        current_day = 1
        for i, meal in enumerate(meal_plan.meals):
            if i % int(meals_per_day) == 0:
                output += f"\n### Day {current_day}\n"
                current_day += 1
            
            output += f"- **{meal.meal_type.title()}:** {meal.recipe_name}\n"
            output += f"  - Prep: {meal.prep_time}min, Cook: {meal.cook_time}min\n"
            output += f"  - Ingredients: {len(meal.ingredients)}\n"
        
        return output
    except Exception as e:
        return f"Error generating meal plan: {str(e)}"


async def generate_shopping_list(days, dietary_restrictions, pantry_items):
    """Generate a shopping list from a meal plan."""
    try:
        restrictions = [r.strip() for r in dietary_restrictions.split(',')] if dietary_restrictions else []
        pantry = [p.strip() for p in pantry_items.split(',')] if pantry_items else []
        
        # First generate meal plan
        preferences = {
            'dietary_restrictions': restrictions,
            'cuisine_preferences': [],
            'meals_per_day': 3
        }
        
        logger.info(f"Generating meal plan for {days} days with preferences: {preferences}")
        
        meal_plan = await meal_agent.generate_meal_plan(
            user_id='gradio_user',
            days=int(days),
            preferences=preferences
        )
        
        logger.info(f"Meal plan generated with {len(meal_plan.meals)} meals")
        
        # Generate shopping list
        shopping_list = await shopping_agent.generate_shopping_list(
            user_id='gradio_user',
            meal_plan=meal_plan,
            pantry=pantry
        )
        
        # Format output
        output = f"# Shopping List: {shopping_list.list_id}\n\n"
        output += f"**Generated from:** {days}-day meal plan\n"
        output += f"**Total Items:** {len(shopping_list.items)}\n"
        output += f"**Categories:** {len(shopping_list.categories)}\n"
        output += f"**Estimated Total:** ${shopping_list.estimated_total:.2f}\n\n"
        
        for category, items in shopping_list.categories.items():
            output += f"\n## {category}\n"
            for item in items:
                output += f"- {item.name}: {item.quantity} {item.unit} (${item.estimated_price:.2f})\n"
        
        return output
    except Exception as e:
        logger.error(f"Error generating shopping list: {str(e)}", exc_info=True)
        return f"Error generating shopping list: {str(e)}\n\nPlease try again or check the logs for more details."


async def plan_trip(destination, days, budget, interests, accommodation_type):
    """Plan a trip."""
    try:
        start_date = date.today() + timedelta(days=30)
        end_date = start_date + timedelta(days=int(days) - 1)
        
        interest_list = [i.strip() for i in interests.split(',')] if interests else []
        
        preferences = {
            'interests': interest_list,
            'accommodation_type': accommodation_type if accommodation_type != "Any" else None,
            'budget_per_night': float(budget) / int(days) if budget else None
        }
        
        trip_plan = await travel_agent.plan_trip(
            user_id='gradio_user',
            destination=destination,
            dates=(start_date.isoformat(), end_date.isoformat()),
            budget=float(budget),
            preferences=preferences
        )
        
        # Format output
        output = f"# Trip Plan: {trip_plan.trip_id}\n\n"
        output += f"**Destination:** {trip_plan.destination}\n"
        output += f"**Duration:** {trip_plan.duration_days()} days\n"
        output += f"**Dates:** {trip_plan.start_date} to {trip_plan.end_date}\n\n"
        
        output += f"## Accommodation\n"
        output += f"**{trip_plan.accommodation.name}** ({trip_plan.accommodation.type})\n"
        output += f"- Location: {trip_plan.accommodation.location}\n"
        output += f"- Cost per night: ${trip_plan.accommodation.cost_per_night:.2f}\n"
        output += f"- Total: ${trip_plan.accommodation.total_cost:.2f}\n"
        output += f"- Amenities: {', '.join(trip_plan.accommodation.amenities)}\n\n"
        
        output += f"## Daily Itinerary\n\n"
        for day in trip_plan.itinerary:
            output += f"### Day {day.day_number} - {day.date}\n"
            
            output += f"\n**Activities:**\n"
            for activity in day.activities:
                output += f"- {activity.name} ({activity.duration}min, ${activity.estimated_cost:.2f})\n"
                output += f"  {activity.description}\n"
            
            output += f"\n**Dining:**\n"
            for restaurant in day.meals:
                output += f"- {restaurant.name} ({restaurant.cuisine_type}, ${restaurant.estimated_cost:.2f})\n"
            
            output += f"\n**Daily Cost:** ${day.total_estimated_cost():.2f}\n\n"
        
        output += f"\n## Total Estimated Cost: ${trip_plan.estimated_cost:.2f}\n"
        
        return output
    except Exception as e:
        return f"Error planning trip: {str(e)}"


async def save_preferences(dietary, cuisine, budget_daily, budget_accommodation, travel_interests):
    """Save user preferences."""
    try:
        user_id = 'gradio_user'
        
        if dietary:
            await memory_bank.save_preference(
                user_id, 
                'dietary_restrictions', 
                [d.strip() for d in dietary.split(',')]
            )
        
        if cuisine:
            await memory_bank.save_preference(
                user_id,
                'cuisine_preferences',
                [c.strip() for c in cuisine.split(',')]
            )
        
        if budget_daily or budget_accommodation:
            budget_prefs = {}
            if budget_daily:
                budget_prefs['daily'] = float(budget_daily)
            if budget_accommodation:
                budget_prefs['accommodation'] = float(budget_accommodation)
            await memory_bank.save_preference(user_id, 'budget_preferences', budget_prefs)
        
        if travel_interests:
            await memory_bank.save_preference(
                user_id,
                'travel_interests',
                [t.strip() for t in travel_interests.split(',')]
            )
        
        return "Preferences saved successfully!"
    except Exception as e:
        return f"Error saving preferences: {str(e)}"


async def load_preferences():
    """Load user preferences."""
    try:
        prefs = await memory_bank.get_all_preferences('gradio_user')
        
        output = "# Your Preferences\n\n"
        
        if prefs.get('dietary_restrictions'):
            output += f"**Dietary Restrictions:** {', '.join(prefs['dietary_restrictions'])}\n"
        
        if prefs.get('cuisine_preferences'):
            output += f"**Cuisine Preferences:** {', '.join(prefs['cuisine_preferences'])}\n"
        
        if prefs.get('budget_preferences'):
            budget = prefs['budget_preferences']
            output += f"**Budget Preferences:**\n"
            if 'daily' in budget:
                output += f"  - Daily: ${budget['daily']:.2f}\n"
            if 'accommodation' in budget:
                output += f"  - Accommodation: ${budget['accommodation']:.2f}\n"
        
        if prefs.get('travel_interests'):
            output += f"**Travel Interests:** {', '.join(prefs['travel_interests'])}\n"
        
        if not prefs:
            output += "*No preferences saved yet.*\n"
        
        return output
    except Exception as e:
        return f"Error loading preferences: {str(e)}"


# Create Gradio interface
with gr.Blocks(title="Personal Life Automation Agent") as app:
    gr.Markdown("""
    # Personal Life Automation Agent System
    ### Google ADK Capstone Project
    
    An intelligent multi-agent system to help you with meal planning, shopping lists, and travel planning.
    """)
    
    with gr.Tabs():
        # Chat Tab
        with gr.Tab("Chat with Agent"):
            gr.Markdown("### Natural Language Interface")
            gr.Markdown("Ask me anything! I can help with meal planning, shopping lists, or travel planning.")
            
            chat_input = gr.Textbox(
                label="Your Message",
                placeholder="e.g., 'Create a 5-day vegetarian meal plan' or 'Plan a trip to Paris'",
                lines=2
            )
            chat_output = gr.Textbox(label="Agent Response", lines=10)
            chat_btn = gr.Button("Send Message", variant="primary")
            
            chat_btn.click(
                fn=lambda msg: asyncio.run(chat_with_agent(msg)),
                inputs=[chat_input],
                outputs=[chat_output]
            )
            
            gr.Examples(
                examples=[
                    ["Create a 3-day meal plan with vegetarian options"],
                    ["Plan a trip to Tokyo for 5 days"],
                    ["I need a shopping list for this week"],
                    ["What can you help me with?"]
                ],
                inputs=[chat_input]
            )
        
        # Meal Planning Tab
        with gr.Tab("Meal Planning"):
            gr.Markdown("### Generate Custom Meal Plans")
            
            with gr.Row():
                meal_days = gr.Slider(1, 14, value=7, step=1, label="Number of Days")
                meal_per_day = gr.Slider(1, 4, value=3, step=1, label="Meals per Day")
            
            meal_dietary = gr.Textbox(
                label="Dietary Restrictions (comma-separated)",
                placeholder="e.g., vegetarian, gluten-free, vegan"
            )
            meal_cuisine = gr.Textbox(
                label="Cuisine Preferences (comma-separated)",
                placeholder="e.g., Italian, Mexican, Japanese"
            )
            
            meal_output = gr.Markdown(label="Meal Plan")
            meal_btn = gr.Button("Generate Meal Plan", variant="primary")
            
            meal_btn.click(
                fn=lambda d, dr, cp, mpd: asyncio.run(generate_meal_plan(d, dr, cp, mpd)),
                inputs=[meal_days, meal_dietary, meal_cuisine, meal_per_day],
                outputs=[meal_output]
            )
        
        # Shopping Tab
        with gr.Tab("Shopping Lists"):
            gr.Markdown("### Generate Shopping Lists from Meal Plans")
            
            shop_days = gr.Slider(1, 7, value=3, step=1, label="Days of Meals")
            shop_dietary = gr.Textbox(
                label="Dietary Restrictions (comma-separated)",
                placeholder="e.g., vegetarian, gluten-free"
            )
            shop_pantry = gr.Textbox(
                label="Pantry Items to Exclude (comma-separated)",
                placeholder="e.g., salt, pepper, olive oil"
            )
            
            shop_output = gr.Markdown(label="Shopping List")
            shop_btn = gr.Button("Generate Shopping List", variant="primary")
            
            shop_btn.click(
                fn=lambda d, dr, p: asyncio.run(generate_shopping_list(d, dr, p)),
                inputs=[shop_days, shop_dietary, shop_pantry],
                outputs=[shop_output]
            )
        
        # Travel Tab
        with gr.Tab("Travel Planning"):
            gr.Markdown("### Plan Your Next Trip")
            
            with gr.Row():
                travel_dest = gr.Textbox(label="Destination", placeholder="e.g., Paris, Tokyo, New York")
                travel_days = gr.Slider(1, 14, value=5, step=1, label="Number of Days")
            
            with gr.Row():
                travel_budget = gr.Number(label="Total Budget ($)", value=2000)
                travel_accommodation = gr.Dropdown(
                    choices=["Any", "hotel", "airbnb", "hostel"],
                    value="Any",
                    label="Accommodation Type"
                )
            
            travel_interests = gr.Textbox(
                label="Interests (comma-separated)",
                placeholder="e.g., museums, food, outdoor, shopping"
            )
            
            travel_output = gr.Markdown(label="Trip Plan")
            travel_btn = gr.Button("Plan Trip", variant="primary")
            
            travel_btn.click(
                fn=lambda dest, days, budget, interests, acc: asyncio.run(
                    plan_trip(dest, days, budget, interests, acc)
                ),
                inputs=[travel_dest, travel_days, travel_budget, travel_interests, travel_accommodation],
                outputs=[travel_output]
            )
    
   

if __name__ == "__main__":
    print("Starting Personal Life Automation Agent System...")
    print("Opening Gradio interface...")
    app.launch(share=False, server_name="0.0.0.0", server_port=7860)


## 🎉 Application Launched!\n\nThe Gradio interface is now running!\nClick the link above to access it.\n\n### Available Features:\n1. **Chat Tab** - Natural language interface\n2. **Meal Planning Tab** - Generate meal plans\n3. **Shopping Lists Tab** - Create shopping lists\n4. **Travel Planning Tab** - Plan trips